# Graph Analyses
Here, we'll perform various analysis by constructing graphs and measure properties of those graphs to learn more about the data

In [2]:
import csv
from scipy.spatial import Delaunay
import numpy as np

In [3]:
# Read in the data
data = open('../data/data.csv', 'r').readlines()
fieldnames = ['x', 'y', 'z', 'unmasked', 'synapses']
reader = csv.reader(data)
reader.next()

rows = [[int(col) for col in row] for row in reader]

# These will come in handy later
sorted_x = sorted(list(set([r[0] for r in rows])))
sorted_y = sorted(list(set([r[1] for r in rows])))
sorted_z = sorted(list(set([r[2] for r in rows])))

In [5]:
sorted_y

[1369,
 1408,
 1447,
 1486,
 1525,
 1564,
 1603,
 1642,
 1681,
 1720,
 1759,
 1798,
 1837,
 1876,
 1915,
 1954,
 1993,
 2032,
 2071,
 2110,
 2149,
 2188,
 2227,
 2266,
 2305,
 2344,
 2383,
 2422,
 2461,
 2500,
 2539,
 2578,
 2617,
 2656,
 2695,
 2734,
 2773,
 2812,
 2851,
 2890,
 2929,
 2968,
 3007,
 3046,
 3085,
 3124,
 3163,
 3202,
 3241,
 3280,
 3319,
 3358]

We'll start with just looking at analysis in euclidian space, then thinking about weighing by synaptic density later. Since we hypothesize that our data will show that tissue varies as we move down the y-axis (z-axis in brain) through cortical layers, an interesting thing to do would be compare properties of the graphs on each layer (ie how does graph connectivity vary as we move through layers).

Let's start by triangulating our data. We'll use Delaunay on each y layer first. Putting our data in the right format for doing graph analysis...

In [59]:
# nested_lst_of_tuples = [tuple(l) for l in rows]
# dtype = [('x','i8'), ('y','i8'), ('z','i8'), ('density','i8'), ('unmasked','i8')]
a = np.array(rows)
b = np.delete(a, np.s_[3::],1)

# Separate layers
b = sorted(b, key=lambda e: e[1])
b = np.array([v.tolist() for v in b])
b = np.split(b, np.where(np.diff(b[:,1]))[0]+1)

In [60]:
b

[array([[  19, 1369,   55],
        [  19, 1369,  166],
        [  19, 1369,  277],
        ..., 
        [4192, 1369,  943],
        [4192, 1369, 1054],
        [4192, 1369, 1165]]), array([[  19, 1408,   55],
        [  19, 1408,  166],
        [  19, 1408,  277],
        ..., 
        [4192, 1408,  943],
        [4192, 1408, 1054],
        [4192, 1408, 1165]]), array([[  19, 1447,   55],
        [  19, 1447,  166],
        [  19, 1447,  277],
        ..., 
        [4192, 1447,  943],
        [4192, 1447, 1054],
        [4192, 1447, 1165]]), array([[  19, 1486,   55],
        [  19, 1486,  166],
        [  19, 1486,  277],
        ..., 
        [4192, 1486,  943],
        [4192, 1486, 1054],
        [4192, 1486, 1165]]), array([[  19, 1525,   55],
        [  19, 1525,  166],
        [  19, 1525,  277],
        ..., 
        [4192, 1525,  943],
        [4192, 1525, 1054],
        [4192, 1525, 1165]]), array([[  19, 1564,   55],
        [  19, 1564,  166],
        [  19, 1564,  277],
 

In [62]:
# Make sure to take 0 and 2 for each centroid when looping

52

In [5]:
graph = Delaunay(centroids)